# Make EGFR Figure 2

This notebooks takes all the trans genes that have positive and negative results and runs a GSEA using Reactome. It then takes a subset of genes from the top hit(Hemostasis) pathway and maps them on a large circle heat map. This heatmap focuses on coagulation and urokinase related genes. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas.util.testing as tm
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import cptac.utils as u
import plot_utils as p 

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  """


# Step 1: Find Trans proteins with opposite effects in different cancers 

Load df with all of the genes that are FDR significant.

In [2]:
FDR_sig = pd.read_csv("Pval_corr_table_Fig_2_return_sig.csv")
FDR_sig = FDR_sig.replace(to_replace ='_proteomics', value = '', regex = True) #Remove proteomics labeling
FDR_sig= FDR_sig.drop(['Unnamed: 0'], axis=1)
FDR_sig = FDR_sig.set_index("Comparison")
FDR_sig = FDR_sig[1:] #Drop EGFR 
FDR_sig

,Correlation_Gbm,P_value_Gbm,Correlation_kidney,P_value_kidney,Correlation_Ovar,P_value_Ovar,Correlation_Brca,P_value_Brca,Correlation_Luad,P_value_Luad,Correlation_hnscc,P_value_hnscc,Correlation_Lscc,P_value_Lscc,Correlation_Colon,P_value_Colon
Comparison,,,,,,,,,,,,,,,,
PHLDA1,0.816848,3.507071e-21,NaN,NaN,NaN,NaN,0.364797,0.002164,NaN,NaN,0.664271,8.888640e-12,0.713420,2.644826e-14,NaN,NaN
GRB2,-0.610889,6.729990e-08,NaN,NaN,NaN,NaN,NaN,NaN,-0.302439,0.020631,-0.532341,3.320092e-06,NaN,NaN,NaN,NaN
CDH4,0.559180,3.420388e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PLA2G15,-0.556624,3.420388e-06,-0.298029,0.02216,NaN,NaN,0.274185,0.016061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOCS2,0.562720,3.420388e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.472624,1.417921e-02,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCDC6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.287735,0.049589
SSR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.301814,0.049589
PSMB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.287839,0.049589


In [3]:
def HasPosNeg(row):
    hasPos = False
    hasNeg= False

    for item in row:
        if pd.isnull(item):
            continue
        if item < 0:
            hasNeg = True
        if item > 0:
            hasPos = True
            
    if hasPos & hasNeg:
        return True
    return False

def CountPosNeg(row):
    hasPos = False
    hasNeg= False
    counter = 0
    for item in row:
        if pd.isnull(item):
            continue
        if item < -0:
            hasNeg = True
            counter += 1
        if item > 0:
            hasPos = True
            counter += 1
    return counter


Subset data frame to include only trans genes that have opposite effects in different cancers by using apply function

In [4]:
col = ["Correlation_Gbm","Correlation_kidney","Correlation_Ovar","Correlation_Brca","Correlation_Luad","Correlation_hnscc","Correlation_Lscc","Correlation_Colon"]
FDR_corr = FDR_sig[col]
FDR_corr["Pos_Neg"] = FDR_corr.apply(HasPosNeg, axis = 1)
FDR_corr_False = FDR_corr[FDR_corr['Pos_Neg']==False]
FDR_corr_False = FDR_corr_False.drop(["Pos_Neg"], axis = 1)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
FDR_corr_False["Num_corr"] = FDR_corr_False.apply(CountPosNeg, axis = 1)
FDR_corr_False.sort_values(by=['Num_corr'],ascending=False)
FDR_corr_False = FDR_corr_False[FDR_corr_False['Num_corr'].isin([2,3,4,5,6])]
FDR_corr_False

,Correlation_Gbm,Correlation_kidney,Correlation_Ovar,Correlation_Brca,Correlation_Luad,Correlation_hnscc,Correlation_Lscc,Correlation_Colon,Num_corr
Comparison,,,,,,,,,
PHLDA1,0.816848,NaN,NaN,0.364797,NaN,0.664271,0.713420,NaN,4
GRB2,-0.610889,NaN,NaN,NaN,-0.302439,-0.532341,NaN,NaN,3
SOCS2,0.562720,NaN,NaN,NaN,NaN,NaN,0.472624,NaN,2
GLA,-0.550491,-0.298348,NaN,NaN,NaN,NaN,NaN,NaN,2
PHLDA3,0.525883,0.432530,NaN,NaN,NaN,0.763784,0.682034,NaN,4
...,...,...,...,...,...,...,...,...,...
S100A14,NaN,NaN,NaN,NaN,NaN,NaN,0.370227,0.351373,2
MOV10,NaN,NaN,NaN,NaN,NaN,NaN,0.302787,0.299218,2
ATP2B1,NaN,NaN,NaN,NaN,NaN,NaN,0.298873,0.346409,2


The manuscript mentions the total number of trans proteins that opposite effects in different cancers. Here is the derivation of that number 

In [6]:
same_sign_prot = FDR_corr_False.index.tolist()
print("Total number of trans proteins with same effects in different cancers is " + str(len(same_sign_prot)))

Total number of trans proteins with same effects in different cancers is 1136


# Run GSEA

In [7]:
same_sign_enr = gp.enrichr(gene_list = same_sign_prot, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='test/enrichr_KEGG')
same_sign_enr.res2d.head(10)

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Arrhythmogenic right ventricular cardiomyopath...,18/74,1.159284e-07,0.000034,0,0,4.282451,68.392004,DSP;ITGA4;JUP;ITGB4;ACTN2;ITGA2;GJA1;CDH2;PKP2...,KEGG_2016
1,Metabolic pathways Homo sapiens hsa01100,114/1239,1.813751e-07,0.000027,0,0,1.619889,25.145046,MSMO1;ENO1;NSDHL;NAMPT;ENPP1;ENPP3;ATP6V1E1;PT...,KEGG_2016
2,Regulation of actin cytoskeleton Homo sapiens ...,32/214,5.044867e-07,0.000049,0,0,2.632618,38.172238,CYFIP2;NCKAP1;ITGB4;WAS;PIK3CD;IQGAP2;ITGAL;FG...,KEGG_2016
3,Biosynthesis of amino acids Homo sapiens hsa01230,17/74,6.249417e-07,0.000046,0,0,4.044537,57.778675,PRPS1;ARG1;PGAM1;GPT;ENO1;PFKL;PKM;CBS;PSAT1;P...,KEGG_2016
4,Glycolysis / Gluconeogenesis Homo sapiens hsa0...,16/67,7.613663e-07,0.000045,0,0,4.204330,59.231243,GPI;ACSS2;ADH1B;PGAM1;ENO1;HK2;LDHA;PFKL;PKM;P...,KEGG_2016
5,Bacterial invasion of epithelial cells Homo sa...,17/78,1.377343e-06,0.000067,0,0,3.837125,51.783367,WAS;FN1;PIK3CD;SEPT1;PIK3CG;SEPT2;CD2AP;SEPT9;...,KEGG_2016
6,B cell receptor signaling pathway Homo sapiens...,16/73,2.588556e-06,0.000108,0,0,3.858769,49.640789,CR2;JUN;SYK;INPPL1;PIK3CD;NFATC2;PIK3CG;VAV1;P...,KEGG_2016
7,Carbon metabolism Homo sapiens hsa01200,20/113,5.248171e-06,0.000192,0,0,3.116041,37.883681,GPI;PRPS1;H6PD;ACSS2;ECHS1;PGAM1;GPT;ENO1;HK2;...,KEGG_2016
8,T cell receptor signaling pathway Homo sapiens...,19/104,5.547202e-06,0.000181,0,0,3.216414,38.925738,JUN;NFATC2;PIK3CD;CD3G;PIK3CG;VAV1;ZAP70;CD4;P...,KEGG_2016
9,Axon guidance Homo sapiens hsa04360,21/127,9.304350e-06,0.000273,0,0,2.911168,33.725962,EPHA4;SEMA4A;SEMA4D;SEMA4B;LIMK1;GNAI3;NFATC2;...,KEGG_2016


In [8]:
#get just the clotting cascade genes and add urokinase genes 
same_sign_df = same_sign_enr.res2d
bcell = same_sign_df.iloc[6,9]
bcell = bcell.split(';')
tcell = same_sign_df.iloc[8,9]
tcell = tcell.split(";")
immune_both =  bcell + tcell
immune = set(immune_both)
immune = list(immune)
btcell_apm = (["CD5","CD8A","BLK","BLNK","CD79A","LYN","CD3E","HLA-DMA","HLA-DMB"])

immune= immune + btcell_apm

#remove = ["CR2","NCK1","CARD11","INPPL1","JUN","KRAS"]
immune.remove("JUN")
immune.remove("KRAS")
immune.remove("CR2")
immune.remove("NCK1")
immune.remove("INPPL1")
immune.remove("CARD11")
immune

ValueError: list.remove(x): x not in list

# Step 3 Make Data frame for figure 2

In [3]:
# Sig PTEN # CD2? CHUK
t_only = ['CD3E', 'CD3G', 'CD4', 'CD8A', 'GRAP2', 'FYB1', 'PRKCQ'] 
b_only = ['PPP3CC', 'CD79A', 'SYK', 'BTK', 'LYN', 'BLNK', 'BLK', 'HLA-DMA', 'HLA-DMB']
both = ['CD2', 'CD5', 'GRB2', 'LCK', 'ZAP70', 'VAV1', 'PTPN6', 'PTPRC', 'PIK3CD', 'PIK3CG', 'INPP5D',
        'PLCG1', 'PLCG2', 'NFATC2', 'PRKCB', 'CHUK', 'NFKB2', 'DOCK2', 'RAC2', 'ELMO1', 'WAS']
immune = t_only + b_only + both

In [4]:
#Get append version of the df with all cancer type, fdr sig trans results
df_FDR_append = pd.read_csv("Pval_corr_table_Fig_2_return_sig_append.csv")
df_FDR_append = df_FDR_append.drop(['Unnamed: 0'], axis=1)

#Remove Proteomics labeling 
df_FDR_append = df_FDR_append.replace(to_replace ='_proteomics', value = '', regex = True) 

#subset dataframe to include genes only desired for figure 
df_FDR_append= df_FDR_append[df_FDR_append.Comparison.isin(immune)]
df_FDR_append['Cancer Type'].unique()
df_FDR_append

,Comparison,Correlation,P_value,Cancer Type
1,GRB2,-0.610889,6.729990e-08,GBM
24,WAS,-0.501918,4.961122e-05,GBM
72,PLCG2,-0.472453,1.145235e-04,GBM
75,SYK,-0.470849,1.202510e-04,GBM
82,CD4,-0.467479,1.362923e-04,GBM
...,...,...,...,...
7781,PRKCQ,-0.284462,3.293176e-02,Head and Neck
7876,RAC2,-0.277100,3.855935e-02,Head and Neck
8461,NFATC2,-0.297428,4.438700e-02,Lscc
8573,PRKCB,0.395927,6.516094e-03,Colon


Set add new column to be unique index and order the new index. This way genes will be grouped by coagulation factors, regulators, and urokinase genes.

In [11]:

df_FDR_append["Index"] = df_FDR_append["Comparison"] + " " + df_FDR_append["Cancer Type"]
df_FDR_append = df_FDR_append.set_index("Index")

ordered_list = []
cancer_list = ['Breast', 'Colon', 'GBM', 'Head and Neck', 'Kidney', 'Lscc', 'Luad']
for gene in immune:
    for cancer in cancer_list:
        ordered_list.append(gene+' '+cancer)

ordered_df = df_FDR_append.reindex(ordered_list)
ordered_df = ordered_df.loc[ordered_df['P_value'] < 0.05] # Keep sig 
                                    
ordered_df

,Comparison,Correlation,P_value,Cancer Type
Index,,,,
CD3E Head and Neck,CD3E,-0.339994,0.020794,Head and Neck
CD3G Head and Neck,CD3G,-0.381032,0.003211,Head and Neck
CD3G Luad,CD3G,-0.333706,0.040933,Luad
CD4 GBM,CD4,-0.467479,0.000136,GBM
CD4 Head and Neck,CD4,-0.417232,0.000566,Head and Neck
...,...,...,...,...
ELMO1 Luad,ELMO1,-0.410098,0.000810,Luad
WAS GBM,WAS,-0.501918,0.000050,GBM
WAS Head and Neck,WAS,-0.503427,0.000016,Head and Neck


# Step 4: Plot Figure 2

In [19]:
legend_min = ordered_df["P_value"].min()
#Make plot using plot utils
p.plotCircleHeatMap(ordered_df, circle_var = "P_value",color_var = "Correlation", x_axis = "Comparison", 
                    y_axis = "Cancer Type", plot_width= 1000, plot_height = 500, legend_min = 1e-8, 
                    legend_max = 0.05, font_size = 12, show_legend = True , save_png = "Figure2.png")

Loading BokehJS ...

In [ ]:
# Sig PTEN # CD2? CHUK
t_only = ['CD3E', 'CD3G', 'CD4', 'CD8A', 'GRAP2', 'FYB1', 'PRKCQ'] 
b_only = ['PPP3CC', 'CD79A', 'SYK', 'BTK', 'LYN', 'BLNK', 'BLK', 'HLA-DMA', 'HLA-DMB']
both = ['CD2', 'CD5', 'GRB2', 'LCK', 'ZAP70', 'VAV1', 'PTPN6', 'PTPRC', 'PIK3CD', 'PIK3CG', 'INPP5D',
        'PLCG1', 'PLCG2', 'NFATC2', 'PRKCB', 'CHUK', 'NFKB2', 'DOCK2', 'RAC2', 'ELMO1', 'WAS']
immune = t_only + b_only + both

# Check if blanks are due to no data 

The follow code chunks show that the following cancers/genes don't have data: colon THBD, Kidney MMP12, and Ovarian MMP12. (As mentioned in EGFR Figure 2 legend)

In [ ]:
#Get append version of the df with all proteins 
df_all_prot_append = pd.read_csv("Pval_corr_table_Fig_2_return_all_append.csv")
df_all_prot_append = df_all_prot_append.drop(['Unnamed: 0'], axis=1)
#Remove Proteomics labeling 
df_all_prot_append = df_all_prot_append.replace(to_replace ='_proteomics', value = '', regex = True) 



In [ ]:
#MHC class2 (CD4) molecules
mhc2 = ["HLA-DPA1","HLA-DPB1","HLA-DMA","HLA-DMB","HLA-DOA","HLA-DOB","HLA-DQ","HLA-DRA","HLA-DRB1"]
#subset dataframe to include genes only desired for figure 
df_all_mhc2 = df_all_prot_append[df_all_prot_append.Comparison.isin(mhc2)]
print("Number of missing columns" )
print(56 - len(df_all_mhc2))
#HLA-DP and HLA-DM missing for all. HLA-DRA and HLA-DRB1 consistently not sig 
df_all_mhc2

In [ ]:
#CD8 and MHC class 1 molecules
mhc1 = ["CD8A","HLA-A","HLA-B","HLA-C"]

df_all_mhc1 = df_all_prot_append[df_all_prot_append.Comparison.isin(mhc1)]
df_all_mhc1

In [ ]:
#Remove Proteomics labeling 
df_all_prot_append = df_all_prot_append.replace(to_replace ='_proteomics', value = '', regex = True) 

#subset dataframe to include genes only desired for figure 
df_all_comp_coag = df_all_prot_append[df_all_prot_append.Comparison.isin(coag_upa)]
print("Number of rows in data frame " + str(len(df_all_comp_coag)))

Our Figure includes 26 genes for 8 cancers. If all data was present there would be 208 rows. However, the data frame only has 205 rows. 3 genes are missing. 

In [ ]:
def find_missing_genes(test_list, full_list):
    for gene in full_list:
        if (gene not in test_list):
            print(gene)
        

In [ ]:
#Get list of genes for colon, kidney, and ovarian
colon = df_all_prot_append[df_all_prot_append["Cancer Type"] == "Colon"]
colon_list = colon.Comparison.to_list()

Kidney = df_all_prot_append[df_all_prot_append["Cancer Type"] == "Kidney"]
Kidney_list = Kidney.Comparison.to_list()

Ovarian = df_all_prot_append[df_all_prot_append["Cancer Type"] == "Ovarian"]
Ovarian_list = Ovarian.Comparison.to_list()

In [ ]:
#Show the 3 missing genes 
print("Ovarian missing genes: ")
find_missing_genes(Ovarian_list, coag_upa)
print("Kidney missing genes: ")
find_missing_genes(Kidney_list, coag_upa)
print("Colon missing genes: ")
find_missing_genes(colon_list, coag_upa)